# LSTM for Sentiment Analysis

Followed [this tutorial](https://pytorch.org/tutorials/beginner/nlp/sequence_models_tutorial.html), and completed it [here](tut/nlp/4_lstm.ipynb)

Helpful blogs:
* [Understanding LSTM Networks](https://colah.github.io/posts/2015-08-Understanding-LSTMs/)
* [The Unreasonable Effectiveness of Recurrent Neural Networks](http://karpathy.github.io/2015/05/21/rnn-effectiveness/)

In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import tokenizers
from torch.nn.utils.rnn import pack_padded_sequence, pad_packed_sequence

import sys
sys.path.append("../..")


class SentimentAnalysisModel(nn.Module):
    def __init__(
        self,
        vocab_size: int,
        emb_dim: int = 30,
        hidden_size: int = 40,
        n_rnn_layers: int = 1,
    ):
        super().__init__()
        self.emb = nn.Embedding(
            num_embeddings=vocab_size,
            embedding_dim=emb_dim,
        )

        self.lstm = nn.LSTM(
            input_size=emb_dim,
            hidden_size=hidden_size,
            num_layers=n_rnn_layers,
        )

        self.seq = nn.Sequential(
            nn.Linear(hidden_size, 500),
            nn.ReLU(),
            nn.Linear(500, 2),
        )

    def forward(self, x: torch.Tensor, lengths: torch.Tensor):


        # x shape: (B, L)
        # convert token indices to embedding values
        x = self.emb(x)
        # x shape: (B, L, Emb dim)

        
        x = x.transpose(0, 1)
        
        # Pack
        x = pack_padded_sequence(x, lengths, enforce_sorted=False)

        _, (h, _) = self.lstm(x)

        return self.seq(h[-1, :, :])

In [ ]:
from tut.sentiment_analysis.helpers import load_sentiment_data
( 
    train_data,
    train_labels,
    train_lengths,
    test_data,
    test_labels,
    test_lengths,
) = load_sentiment_data()

In [ ]:
# load in tokenizer
tokenizer = tokenizers.Tokenizer.from_file("models/tokenizer.json")
vocab_size = tokenizer.get_vocab_size()


In [ ]:
device = "cuda" if torch.cuda.is_available() else "cpu"
model = SentimentAnalysisModel(vocab_size=vocab_size).to(device)

optimizer = torch.optim.Adam(model.parameters(), lr=0.01)
loss_function = F.cross_entropy

# Training
from tqdm import tqdm

epochs = 10
batch_size = 1000
eval_batch_size = 1000


def _calc_accuracy(
    data: torch.Tensor,
    labels: torch.Tensor,
    lengths: torch.Tensor,
    batch_size: torch.Tensor,
):
    cum_loss = 0
    correct = 0
    total = 0
    with torch.no_grad():
        for i in tqdm(range(0, len(data), batch_size)):
            input = data[i : i + batch_size].to(device)
            _labels = labels[i : i + batch_size].to(device)
            _lengths = lengths[i : i + batch_size]

            output = model(input, _lengths)
            loss = loss_function(output, _labels)

            cum_loss += loss
            _, y_pred = torch.max(output, dim=1)
            correct += sum(_labels == y_pred)
            total += len(_labels)

    return correct, total, cum_loss


for epoch in range(epochs):
    print(epoch, epochs)
    test_correct, test_total, test_loss = _calc_accuracy(
        test_data, test_labels, test_lengths, batch_size=eval_batch_size
    )
    train_correct, train_total, train_loss = _calc_accuracy(
        train_data, train_labels, train_lengths, batch_size=eval_batch_size
    )

    print(f"test: {test_correct / test_total: 0.2f}, {test_loss}")
    print(f"train: {train_correct / train_total: 0.2f}, {train_loss}")

    for i in tqdm(range(0, len(train_data), batch_size)):
        input = train_data[i : i + batch_size].to(device)
        labels = train_labels[i : i + batch_size].to(device)
        lengths = train_lengths[i : i + batch_size]

        optimizer.zero_grad()
        output = model(input, lengths)
        loss = loss_function(output, labels)
        loss.backward()
        optimizer.step()


0 10


100%|██████████| 40/40 [00:04<00:00,  9.54it/s]


test:  0.50, 6.932585716247559
train:  0.50, 27.736610412597656


 56%|█████▋    | 225/400 [00:18<00:13, 13.12it/s]